## Analisando Dados do Uber com Spark

Dataset: https://github.com/fivethirtyeight/uber-tlc-foil-response

Esse conjunto de dados contém dados de mais e 4,5 milhoes de captações Uber na cidade de NovaYork, de abril a setembro de 2014 a 14,3 milhões de captações Uber de janeiro a junho de 2015. Dados em nível de viagem sobre 10 outras empresas de veículos de aluguel (FHV) bem como dados agregados para 329 empresas de FHV, também estão incluídos. Todos os arquivos foram recebidos em 3 de agosto, 15 de setembro e 22 de setembro de 2015

1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

2- Qual o total de veículos que passam pela base B02617?

3- Qual o total de corridas por base? Apresente de forma decrescente.

In [2]:
# Carregando os dados no Spark
uberRDD = sc.textFile("data/uber.csv")

In [3]:
# Verificando o tipo de dados criado
type(uberRDD)

pyspark.rdd.RDD

In [4]:
# Verificando o numero de registros
uberRDD.count()

355

In [5]:
# Visualizando as colunas do conjunto de dados
uberRDD.take(1)

['dispatching_base_number,date,active_vehicles,trips']

In [7]:
# Visualizando as primeiras linhas do conjunto de dados
uberRDD.take(15)

['dispatching_base_number,date,active_vehicles,trips',
 'B02512,1/1/2015,190,1132',
 'B02765,1/1/2015,225,1765',
 'B02764,1/1/2015,3427,29421',
 'B02682,1/1/2015,945,7679',
 'B02617,1/1/2015,1228,9537',
 'B02598,1/1/2015,870,6903',
 'B02598,1/2/2015,785,4768',
 'B02617,1/2/2015,1137,7065',
 'B02512,1/2/2015,175,875',
 'B02682,1/2/2015,890,5506',
 'B02765,1/2/2015,196,1001',
 'B02764,1/2/2015,3147,19974',
 'B02765,1/3/2015,201,1526',
 'B02617,1/3/2015,1188,10664']

In [8]:
uberRDDSplit = uberRDD.map(lambda x: x.split(","))
uberRDDSplit.collect()

[['dispatching_base_number', 'date', 'active_vehicles', 'trips'],
 ['B02512', '1/1/2015', '190', '1132'],
 ['B02765', '1/1/2015', '225', '1765'],
 ['B02764', '1/1/2015', '3427', '29421'],
 ['B02682', '1/1/2015', '945', '7679'],
 ['B02617', '1/1/2015', '1228', '9537'],
 ['B02598', '1/1/2015', '870', '6903'],
 ['B02598', '1/2/2015', '785', '4768'],
 ['B02617', '1/2/2015', '1137', '7065'],
 ['B02512', '1/2/2015', '175', '875'],
 ['B02682', '1/2/2015', '890', '5506'],
 ['B02765', '1/2/2015', '196', '1001'],
 ['B02764', '1/2/2015', '3147', '19974'],
 ['B02765', '1/3/2015', '201', '1526'],
 ['B02617', '1/3/2015', '1188', '10664'],
 ['B02598', '1/3/2015', '818', '7432'],
 ['B02682', '1/3/2015', '915', '8010'],
 ['B02512', '1/3/2015', '173', '1088'],
 ['B02764', '1/3/2015', '3215', '29729'],
 ['B02512', '1/4/2015', '147', '791'],
 ['B02682', '1/4/2015', '812', '5621'],
 ['B02598', '1/4/2015', '746', '5223'],
 ['B02765', '1/4/2015', '183', '993'],
 ['B02617', '1/4/2015', '1088', '7729'],
 ['B02

### 1 - Quantos são e quais sao as bases de carros da uber?

In [27]:
# Numero de bases de carros do Uber
print("Número de Bases de carros:", uberRDDSplit.map(lambda linha: linha[0]).distinct().count()-1)

Número de Bases de carros: 6


In [55]:
# Quais sao as bases de carros
uberRDDSplit.map(lambda linha: linha[0]).distinct().collect()

['dispatching_base_number',
 'B02765',
 'B02682',
 'B02598',
 'B02512',
 'B02764',
 'B02617']

### 2- Qual o total de veículos que passam pela base B02617?

In [51]:
print("Carros que passaram pela base B02617:", uberRDDSplit.filter(lambda x: "B02617" in x).count())

Carros que passaram pela base B02617: 59


In [56]:
# Gravando os dados dos vaiculos da base B02617 em um novo RDD
b02617 = uberRDDSplit.filter(lambda x: "B02617" in x)

In [59]:
# Total de dias em que o numero de corridas foi superior a 16000
b02617.filter(lambda linha: int(linha[3]) > 16000).count()

4

In [60]:
# Dias em que o numero de corridas foi superior a 16000
b02617.filter(lambda linha: int(linha[3]) > 16000).collect()

[['B02617', '2/13/2015', '1590', '16996'],
 ['B02617', '2/14/2015', '1486', '16999'],
 ['B02617', '2/20/2015', '1574', '16856'],
 ['B02617', '2/21/2015', '1443', '16098']]

### 3- Qual o total de corridas por base? Apresente de forma decrescente.

In [61]:
uberRDD2 = sc.textFile("data/uber.csv").filter(lambda line: "base" not in line).map(lambda line: line.split(","))

In [62]:
# Aplicando redução para calcular o total por base
uberRDD2.map(lambda kp: (kp[0], int(kp[3]))).reduceByKey(lambda k,v: k + v).collect()

[('B02765', 193670),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02512', 93786),
 ('B02764', 1914449),
 ('B02617', 725025)]

In [63]:
# Ordem decrescente
uberRDD2.map(lambda kp: (kp[0], int(kp[3]))).reduceByKey(lambda k,v: k + v).takeOrdered(10, key = lambda x: -x[1])

[('B02764', 1914449),
 ('B02617', 725025),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02765', 193670),
 ('B02512', 93786)]